In [ ]:

# multimodal_stimulus_fmri_predict/core/classifier_factory.py
from typing import Dict, Any
from .base_classifier import BaseClassifier
from ..classifiers.vision_transformer import VisionTransformerClassifier
from ..classifiers.resnet import ResNetClassifier
from ..classifiers.efficientnet import EfficientNetClassifier
from ..classifiers.multimodal_classifier import MultiModalClassifier

class ClassifierFactory:
    """Factory class for creating different classifier instances."""
    
    _classifiers = {
        'vit': VisionTransformerClassifier,
        'resnet': ResNetClassifier,
        'efficientnet': EfficientNetClassifier,
        'multimodal': MultiModalClassifier
    }
    
    @classmethod
    def create_classifier(cls, classifier_type: str, config: Dict[str, Any]) -> BaseClassifier:
        """Create a classifier instance based on type and config."""
        if classifier_type not in cls._classifiers:
            raise ValueError(f"Unknown classifier type: {classifier_type}. "
                           f"Available types: {list(cls._classifiers.keys())}")
        
        classifier_class = cls._classifiers[classifier_type]
        return classifier_class(config)
    
    @classmethod
    def register_classifier(cls, name: str, classifier_class: type):
        """Register a new classifier type."""
        cls._classifiers[name] = classifier_class
    
    @classmethod
    def available_classifiers(cls) -> list:
        """Get list of available classifier types."""
        return list(cls._classifiers.keys())


# multimodal_stimulus_fmri_predict/utils/experiment_runner.py
import json
import os
from typing import Dict, List, Any
import torch
from torch.utils.data import DataLoader
import pandas as pd
from ..core.classifier_factory import ClassifierFactory
from ..core.base_classifier import BaseClassifier
import time

class ExperimentRunner:
    """Class to run experiments with different classifiers and configurations."""
    
    def __init__(self, results_dir: str = "results"):
        self.results_dir = results_dir
        os.makedirs(results_dir, exist_ok=True)
    
    def run_experiment(self, experiment_config: Dict[str, Any], 
                      train_loader: DataLoader, val_loader: DataLoader,
                      test_loader: DataLoader) -> Dict[str, Any]:
        """Run a single experiment with given configuration."""
        
        classifier_type = experiment_config['classifier_type']
        classifier_config = experiment_config['classifier_config']
        training_config = experiment_config.get('training_config', {})
        
        print(f"Running experiment with {classifier_type} classifier...")
        
        # Create classifier
        classifier = ClassifierFactory.create_classifier(classifier_type, classifier_config)
        
        # Training
        start_time = time.time()
        epochs = training_config.get('epochs', 10)
        history = classifier.train(train_loader, val_loader, epochs)
        training_time = time.time() - start_time
        
        # Evaluation
        test_loss, test_acc = classifier.evaluate(test_loader)
        
        # Compile results
        results = {
            'classifier_type': classifier_type,
            'classifier_config': classifier_config,
            'training_config': training_config,
            'history': history,
            'test_loss': test_loss,
            'test_accuracy': test_acc,
            'training_time': training_time,
            'final_val_accuracy': history['val_acc'][-1] if history['val_acc'] else 0
        }
        
        print(f"Experiment completed. Test accuracy: {test_acc:.4f}")
        return results
    
    def run_multiple_experiments(self, experiment_configs: List[Dict[str, Any]], 
                               train_loader: DataLoader, val_loader: DataLoader,
                               test_loader: DataLoader) -> pd.DataFrame:
        """Run multiple experiments and return results as DataFrame."""
        
        all_results = []
        
        for i, config in enumerate(experiment_configs):
            print(f"\n--- Experiment {i+1}/{len(experiment_configs)} ---")
            try:
                results = self.run_experiment(config, train_loader, val_loader, test_loader)
                all_results.append(results)
                
                # Save intermediate results
                self.save_results(results, f"experiment_{i+1}")
                
            except Exception as e:
                print(f"Experiment {i+1} failed: {str(e)}")
                continue
        
        # Create summary DataFrame
        summary_data = []
        for result in all_results:
            summary_data.append({
                'classifier_type': result['classifier_type'],
                'test_accuracy': result['test_accuracy'],
                'final_val_accuracy': result['final_val_accuracy'],
                'training_time': result['training_time'],
                'config': str(result['classifier_config'])
            })
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv(os.path.join(self.results_dir, "experiment_summary.csv"), index=False)
        
        return summary_df
    
    def save_results(self, results: Dict[str, Any], filename: str):
        """Save experiment results to JSON file."""
        filepath = os.path.join(self.results_dir, f"{filename}.json")
        with open(filepath, 'w') as f:
            json.dump(results, f, indent=2, default=str)


# multimodal_stimulus_fmri_predict/configs/experiment_configs.py
"""
Configuration templates for different experimental setups.
"""

def get_vit_configs():
    """Get Vision Transformer experiment configurations."""
    return [
        {
            'classifier_type': 'vit',
            'classifier_config': {
                'pretrained': True,
                'image_size': 224,
                'patch_size': 16,
                'num_classes': 2,
                'learning_rate': 1e-4
            },
            'training_config': {'epochs': 20}
        },
        {
            'classifier_type': 'vit',
            'classifier_config': {
                'pretrained': True,
                'image_size': 224,
                'patch_size': 32,
                'num_classes': 2,
                'learning_rate': 5e-5
            },
            'training_config': {'epochs': 20}
        }
    ]

def get_resnet_configs():
    """Get ResNet experiment configurations."""
    return [
        {
            'classifier_type': 'resnet',
            'classifier_config': {
                'model_name': 'resnet50',
                'pretrained': True,
                'num_classes': 2,
                'learning_rate': 1e-4
            },
            'training_config': {'epochs': 15}
        },
        {
            'classifier_type': 'resnet',
            'classifier_config': {
                'model_name': 'resnet101',
                'pretrained': True,
                'num_classes': 2,
                'learning_rate': 5e-5
            },
            'training_config': {'epochs': 15}
        }
    ]

def get_multimodal_configs():
    """Get multi-modal experiment configurations."""
    return [
        {
            'classifier_type': 'multimodal',
            'classifier_config': {
                'image_backbone': 'resnet50',
                'fmri_input_dim': 1000,
                'fusion_dim': 256,
                'num_classes': 2,
                'learning_rate': 1e-4
            },
            'training_config': {'epochs': 25}
        },
        {
            'classifier_type': 'multimodal',
            'classifier_config': {
                'image_backbone': 'efficientnet_b0',
                'fmri_input_dim': 1000,
                'fusion_dim': 512,
                'num_classes': 2,
                'learning_rate': 5e-5
            },
            'training_config': {'epochs': 25}
        }
    ]